In [1]:
%%configure
{
    "defaultLakehouse": {  
        "name": "{{lakehouse_name}}",
    }
}

StatementMeta(, e0499f6a-53c8-4758-bcbe-2f8e1b5e1870, -1, Finished, Available, Finished)

In [2]:
# Created at: {{createddate}}
metadataTablename = "comparemetadata"

StatementMeta(, bb2596c2-3670-48d5-8508-ccbe213ca3af, 3, Finished, Available, Finished)

In [13]:

from pyspark.sql.types import StructType, StructField, StringType
from pyspark.sql.functions import *
import time
# This method gets all table columns for all tables in a database
def get_all_table_columns(database):
  
  schema = StructType([ \
    StructField('database', StringType(), True),\
    StructField('table_name', StringType(), True),\
    StructField('col_name', StringType(), True),\
    StructField('data_type', StringType(), True),\
    StructField('comment', StringType(), True)\
  ])  
  
  dfcols = spark.createDataFrame(spark.sparkContext.emptyRDD(), schema)  
  dftabs = spark.sql('show tables from {}'.format(database))
  
  print('{} tables in {} database'.format(dftabs.count(), database))  
  # for each table collect the columns and append 
  for row in dftabs.rdd.collect():
    dfcol_tmp = spark.sql('desc table {}.{}'.format(row[0], row[1]))
    
    dfcol_tmp = dfcol_tmp.select(\
                                  lit(row[0]).alias('database'), \
                                  lit(row[1]).alias('table_name'), \
                                  'col_name', \
                                  'data_type', \
                                  'comment'\
                                ) 
    dfcols = dfcols.union(dfcol_tmp)  
  
  print('{} columns in {} database'.format(dfcols.count(),database))
  
  return dfcols

def buildMetadataTable(lakehouseName):
    spark.sql(f"DROP TABLE {metadataTablename}")
    dfMeta = get_all_table_columns(lakehouseName)
    dfMeta.write.format("delta").saveAsTable(metadataTablename)
    return dfMeta

StatementMeta(, bb2596c2-3670-48d5-8508-ccbe213ca3af, 29, Finished, Available, Finished)

In [7]:
display(buildMetadataTable('{{lakehouse_name}}'))

StatementMeta(, bb2596c2-3670-48d5-8508-ccbe213ca3af, 13, Finished, Available, Finished)

5 tables in avtest02noschema database
17 columns in avtest02noschema database


SynapseWidget(Synapse.DataFrame, 748c0f8c-d681-4272-9fa5-5932aa892874)